In [1]:
import itertools

import polars as pl

In [19]:
YEAR_MONTHS = list(itertools.product((1, 2),list(range(1, 13))))

In [20]:
files = [
    f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_202{i}-{j}.parquet"
    if j > 9
    else f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_202{i}-0{j}.parquet"
    for i,j in YEAR_MONTHS  # Dec not available
]

In [22]:
%%time

df = pl.concat(
    [pl.read_parquet(url) for url in files[:-1]]
)

CPU times: user 24.9 s, sys: 7.72 s, total: 32.6 s
Wall time: 1min 2s


In [23]:
df.estimated_size('gb')

9.606467684730887

In [24]:
df.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.1,1.0,"""N""",142,43,2,8.0,3.0,0.5,0.0,0.0,0.3,11.8,2.5,null
1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.2,1.0,"""N""",238,151,2,3.0,0.5,0.5,0.0,0.0,0.3,4.3,0.0,null
1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.7,1.0,"""N""",132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,null
1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.6,1.0,"""N""",138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,null
2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,"""N""",68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,null


In [25]:
%%time

(
    df.with_columns(
        is_congested=pl.col('congestion_surcharge').fill_null(0) > 0
    )
    .groupby('DOLocationID')
    .agg(
        mean_fare=pl.mean('fare_amount'),
        count=pl.count(),
        congestion=pl.mean('is_congested')
    )
    .sort('count', descending=True)
)

CPU times: user 3.37 s, sys: 1.1 s, total: 4.46 s
Wall time: 2.26 s


DOLocationID,mean_fare,count,congestion
i64,f64,u32,f64
236,9.832427,2982823,0.962186
237,9.302696,2830014,0.971781
161,10.925757,2321912,0.965709
170,10.941102,1992061,0.962805
239,10.95057,1886866,0.964539
141,10.147934,1880508,0.965594
142,10.46468,1842501,0.96582
48,12.619569,1759038,0.959775
162,11.032523,1728463,0.966986
